# Кодирование регионов

In [50]:
import pandas as pd

df = pd.read_excel(r"C:\Users\Qawse\Desktop\ВКР\Спрос\procurement.xlsx")
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')
df['end_date'] = pd.to_datetime(df['end_date'], format='%d.%m.%Y')
df['is_hidden'] = df['is_hidden'].map({'нет': 0, 'да': 1})
df = df[df['end_date'] <= '2024-12-31']
df = df.drop(['is_hidden', 'responsible', 'phone', 'mail'], axis = 1)
df

  0%|                                                                              | 4/3455 [02:30<36:09:25, 37.72s/it]


,tender,reason,customer,start_price,region,start_date,end_date,number
0,OZON fresh. Установка холодильного оборудовани...,OZON fresh. Установка холодильного оборудовани...,"ООО ""ИНТЕРНЕТ РЕШЕНИЯ""",NaN,Санкт-Петербург,2024-12-02,2024-12-16,169-990
2,Взрывозащищенный холодильный агрегат (Сплит-си...,Взрывозащищенный холодильный агрегат (сплит-си...,"ООО ""ЗАВОД ПРОМЫШЛЕННОГО ОБОРУДОВАНИЯ""",NaN,Челябинская обл г Челябинск,2024-09-24,2024-10-03,152-882
6,Выполнение ремонтно-восстановительных работ хо...,ремонтно-восстановительные работы холодильных ...,"ОАО ""АВЕКСИМА""",NaN,Московская обл г Химки,2024-12-16,2024-12-19,173-895
7,Выбор подрядной организации на поставку инжене...,Выбор подрядной организации на поставку инжене...,"ООО ""СМАРТ КОНСТРАКШН""",NaN,Москва,2024-06-04,2024-07-15,SBR028-2406040026
12,Закупка у единственного поставщика (подрядчика...,Компрессорный холодильный агрегат,"МАУ ДО ""СШОР ""ОРЛЕНОК"" Г. ПЕРМИ",1269999.96,Пермский край,2024-12-26,2024-12-26,32414392653
...,...,...,...,...,...,...,...,...
3464,2100-K01-К-11-01033-2020 «Электротехническая п...,Агрегат холодильный Rittal SK 3304.500,"ООО""ТРАНСНЕФТЬ - ВОСТОК""",73874179.04,Иркутская обл г Братск,2020-01-20,2020-02-10,32008777726
3465,Установка охл.,УСТАНОВКА ХОЛОДИЛЬНАЯ ВМТ-КСИРОН-1 КСИРОН-ХОЛОД,"АО ""МСЗ""",NaN,Московская обл г. Электросталь,2020-01-14,2020-01-17,1421975_38
3466,Установка охл.,УСТАНОВКА ХОЛОДИЛЬНАЯ ВМТ-КСИРОН-1 КСИРОН-ХОЛОД,"АО ""МСЗ""",NaN,Москва,2020-01-14,2020-01-17,2629168
3467,"ГАЗ 2775-01, автофургон изотермический, 2004 г...","ГАЗ 2775-01, автофургон изотермический, 2004 г...","ООО ""САРАТОВГАЗТОРГ""",160000.00,Саратовская обл г Саратов,2020-01-10,2020-02-07,ГП001389


In [53]:
import re

def standardize_region(region):
    if pd.isna(region) or region.strip() in ('', ' '):
        return None
    
    # Удаляем лишние пробелы и точки
    region = re.sub(r'\s+', ' ', region.strip())
    
    # Заменяем сокращения
    region = (
        region.replace("обл ", "область, ")
              .replace("Респ ", "Республика ")
              .replace("Респ.", "Республика ")
              .replace("АО ", "автономный округ ")
              .replace("рп ", "рабочий посёлок ")
              .replace("пгт. ", "посёлок городского типа ")
              .replace("пгт ", "посёлок городского типа ")
              .replace("с/п ", "сельское поселение ")
              .replace("тер ", "территория ")
    )
    
    # Добавляем "г." перед названиями городов, если его нет
    if 'г ' in region and 'г.' not in region:
        region = region.replace('г ', 'г. ')
    
    # Убираем дублирование "г." (например, "г. г. Москва")
    region = re.sub(r'г\.\s*г\.', 'г.', region)
    
    # Если запись содержит "область" и город, добавляем запятую
    if 'область' in region and 'г.' in region and ',' not in region:
        region = region.replace('г.', ', г.')
    
    return region

df['region_clean'] = df['region'].apply(standardize_region)

In [54]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
from tqdm import tqdm  # для прогресс-бара (опционально)

# Инициализация геокодера
geolocator = Nominatim(user_agent="region_map_app", timeout=10)

# Словарь для кэширования координат (чтобы не запрашивать повторно)
coordinates = {}

def geocode_region(region):
    """Функция для геокодирования региона с обработкой ошибок"""
    if pd.isna(region) or not region.strip():
        return None
    
    if region in coordinates:
        return coordinates[region]
    
    try:
        location = geolocator.geocode(f"{region}, Россия")
        if location:
            coords = (location.latitude, location.longitude)
            coordinates[region] = coords
            return coords
        else:
            # Попробуем без указания страны
            location = geolocator.geocode(region)
            if location:
                coords = (location.latitude, location.longitude)
                coordinates[region] = coords
                return coords
            return None
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Ошибка для региона '{region}': {str(e)}")
        time.sleep(1)
        return None

# Применяем геокодирование с прогресс-баром
print("Начинаем геокодирование регионов...")
df['coords'] = [geocode_region(region) for region in tqdm(df['region_clean'])]

# Анализ результатов
success_rate = df['coords'].notna().mean()
print(f"\nУспешно геокодировано: {success_rate:.1%} регионов")


Начинаем геокодирование регионов...


100%|██████████████████████████████████████████████████████████████████████████████| 3455/3455 [11:33<00:00,  4.98it/s]



Успешно геокодировано: 95.3% регионов


  0%|▏                                                                            | 11/3455 [11:58<62:26:57, 65.28s/it]


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Qawse\\Desktop\\ВКР\\Спрос\\tenders_with_coordinates.xlsx'

In [55]:
df.to_excel(r"C:\Users\Qawse\Desktop\ВКР\Спрос\tenders_with_coordinates.xlsx")
print("Файл успешно сохранен по пути: C:\\Users\\Qawse\\Desktop\\ВКР\\Спрос\\tenders_with_coordinates.xlsx")

Файл успешно сохранен по пути: C:\Users\Qawse\Desktop\ВКР\Спрос\tenders_with_coordinates.xlsx


In [64]:
df[df.coords.isna()==True].region_clean.unique()

array(['Удмуртская Респ', 'Татарстан Респ', None,
       'Краснодарский край посёлок городского типа Сириус',
       'Донецкая Народная респ. г. Мариуполь', 'Крым Респ',
       'Мордовия Респ', 'Новосибирская область, рабочий посёлок Кольцово',
       'Башкортостан Респ', 'Бурятия Респ',
       'Брянская область, территория Рамасухское городское поселение',
       'Калининградская область, посёлок городского типа Янтарный'],
      dtype=object)

In [70]:
import pandas as pd
from ast import literal_eval

manual_coords = {
    'Удмуртская Респ': (57.0671, 53.0273),  # Ижевск
    'Татарстан Респ': (55.7963, 49.1088),    # Казань
    'Краснодарский край посёлок городского типа Сириус': (43.4021, 39.9633),
    'Донецкая Народная респ. г. Мариуполь': (47.0971, 37.5434),
    'Крым Респ': (44.9521, 34.1024),         # Симферополь
    'Мордовия Респ': (54.1805, 45.1862),     # Саранск
    'Новосибирская область, рабочий посёлок Кольцово': (54.9375, 83.1866),
    'Башкортостан Респ': (54.7351, 55.9587), # Уфа
    'Бурятия Респ': (51.8335, 107.5841),     # Улан-Удэ
    'Брянская область, территория Рамасухское городское поселение': (53.2436, 34.3637),
    'Калининградская область, посёлок городского типа Янтарный': (54.8716, 19.9333)
}

# Функция для безопасного преобразования координат
def safe_convert(coords):
    try:
        return literal_eval(str(coords)) if pd.notna(coords) else None
    except:
        return None

# Применяем ручные координаты только к указанным регионам
for region, coords in manual_coords.items():
    mask = (df['region_clean'] == region) & (df['coords'].isna())
    df.loc[mask, 'coords'] = str(coords)

# Преобразуем координаты (оставляем NaN где не получилось)
df['coords'] = df['coords'].apply(safe_convert)

df.to_excel('C:\\Users\\Qawse\\Desktop\\ВКР\\Спрос\\tenders_with_coordinates.xlsx', index=False)

print(f"Процент заполненных координат: {df['coords'].notna().mean():.1%}")

Файл сохранён: None
Процент заполненных координат: 97.7%


# Загрузка данных

In [21]:
import pandas as pd

df = pd.read_excel(r"C:\Users\Qawse\Desktop\Кайрос\Спрос\regional_distribution.xlsx")
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')
df['end_date'] = pd.to_datetime(df['end_date'], format='%d.%m.%Y')
df = df[df['end_date'] <= '2024-12-31']
df

,tender,reason,customer,start_price,region,start_date,end_date,number,region_clean,coords
0,OZON fresh. Установка холодильного оборудовани...,OZON fresh. Установка холодильного оборудовани...,"ООО ""ИНТЕРНЕТ РЕШЕНИЯ""",NaN,Санкт-Петербург,2024-12-02,2024-12-16,169-990,Санкт-Петербург,"(59.9606739, 30.1586551)"
1,Взрывозащищенный холодильный агрегат (Сплит-си...,Взрывозащищенный холодильный агрегат (сплит-си...,"ООО ""ЗАВОД ПРОМЫШЛЕННОГО ОБОРУДОВАНИЯ""",NaN,Челябинская обл г Челябинск,2024-09-24,2024-10-03,152-882,"Челябинская область, г. Челябинск","(56.2210938, 60.8732388)"
2,Выполнение ремонтно-восстановительных работ хо...,ремонтно-восстановительные работы холодильных ...,"ОАО ""АВЕКСИМА""",NaN,Московская обл г Химки,2024-12-16,2024-12-19,173-895,"Московская область, г. Химки","(55.9423557, 37.3472703)"
3,Выбор подрядной организации на поставку инжене...,Выбор подрядной организации на поставку инжене...,"ООО ""СМАРТ КОНСТРАКШН""",NaN,Москва,2024-06-04,2024-07-15,SBR028-2406040026,Москва,"(55.625578, 37.6063916)"
4,Закупка у единственного поставщика (подрядчика...,Компрессорный холодильный агрегат,"МАУ ДО ""СШОР ""ОРЛЕНОК"" Г. ПЕРМИ",1269999.96,Пермский край,2024-12-26,2024-12-26,32414392653,Пермский край,"(58.5951603, 56.3159546)"
...,...,...,...,...,...,...,...,...,...,...
3450,2100-K01-К-11-01033-2020 «Электротехническая п...,Агрегат холодильный Rittal SK 3304.500,"ООО""ТРАНСНЕФТЬ - ВОСТОК""",73874179.04,Иркутская обл г Братск,2020-01-20,2020-02-10,32008777726,"Иркутская область, г. Братск","(56.1228365, 101.5987288)"
3451,Установка охл.,УСТАНОВКА ХОЛОДИЛЬНАЯ ВМТ-КСИРОН-1 КСИРОН-ХОЛОД,"АО ""МСЗ""",NaN,Московская обл г. Электросталь,2020-01-14,2020-01-17,1421975_38,"Московская область, г. Электросталь","(55.7671289, 38.3791098)"
3452,Установка охл.,УСТАНОВКА ХОЛОДИЛЬНАЯ ВМТ-КСИРОН-1 КСИРОН-ХОЛОД,"АО ""МСЗ""",NaN,Москва,2020-01-14,2020-01-17,2629168,Москва,"(55.625578, 37.6063916)"
3453,"ГАЗ 2775-01, автофургон изотермический, 2004 г...","ГАЗ 2775-01, автофургон изотермический, 2004 г...","ООО ""САРАТОВГАЗТОРГ""",160000.00,Саратовская обл г Саратов,2020-01-10,2020-02-07,ГП001389,"Саратовская область, г. Саратов","(52.0054416, 47.8034362)"


# Дубликаты

In [22]:
import pandas as pd

df_deduped = df.drop_duplicates(
    subset=['number', 'tender', 'end_date'],  # Можно добавить 'start_date'
    keep='first'  # или 'last' для сохранения последней версии
)

print(f"Исходных записей: {len(df)}")
print(f"После удаления дубликатов: {len(df_deduped)}")
print(f"Удалено записей: {len(df) - len(df_deduped)}")
df_deduped = df

Исходных записей: 3455
После удаления дубликатов: 3421
Удалено записей: 34


In [23]:
import pandas as pd
import numpy as np

# Проверка пустых значений в колонке region
def check_empty_values(df):
    # Считаем явные NaN
    nan_count = df['region'].isna().sum()
    
    # Считаем пустые строки
    empty_str_count = (df['region'] == '').sum()
    
    # Считаем строки только с пробелами
    whitespace_count = df['region'].str.strip().eq('').sum()
    
    # Считаем специальные "пустые" значения (например, ' ', 'null', 'NA')
    special_empty_count = df['region'].isin([' ', 'null', 'NA', 'N/A']).sum()
    
    print(f"""
    Анализ пустых значений в колонке 'region':
    - Явные NaN: {nan_count}
    - Пустые строки (''): {empty_str_count}
    - Только пробелы: {whitespace_count}
    - Специальные пустые значения: {special_empty_count}
    - Всего пустышек: {nan_count + empty_str_count + whitespace_count + special_empty_count}
    """)

# Приведение к стандартному формату пустых значений
def clean_empty_values(df):
    # Заменяем разные варианты пустых значений на единый стандарт NaN
    df['region'] = df['region'].replace(['', ' ', 'null', 'NA', 'N/A'], np.nan)
    return df

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tender        3455 non-null   object        
 1   reason        3455 non-null   object        
 2   customer      3103 non-null   object        
 3   start_price   2170 non-null   float64       
 4   region        3455 non-null   object        
 5   start_date    3455 non-null   datetime64[ns]
 6   end_date      3455 non-null   datetime64[ns]
 7   number        3455 non-null   object        
 8   region_clean  3376 non-null   object        
 9   coords        3376 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 270.1+ KB


# Карта

In [6]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from ast import literal_eval

df['coords'] = df['coords'].apply(lambda x: literal_eval(x) if pd.notna(x) and x.startswith('(') else None)

# Создаем базовую карту с центром в России
m = folium.Map(location=[60, 90], zoom_start=4, tiles='CartoDB positron')

# Добавляем тепловую карту для визуализации плотности контрактов
heat_data = [[coord[0], coord[1]] for coord in df['coords'].dropna() if len(coord) == 2]
HeatMap(heat_data, radius=15).add_to(m)

# Добавляем маркеры для каждого контракта
marker_cluster = folium.plugins.MarkerCluster().add_to(m)

for idx, row in df.dropna(subset=['coords']).iterrows():
    folium.Marker(
        location=row['coords'],
        popup=folium.Popup(
            f"<b>Контракт:</b> {row['tender'][:100]}...<br>"
            f"<b>Заказчик:</b> {row['customer']}<br>"
            f"<b>Регион:</b> {row['region_clean']}",
            max_width=300
        ),
        icon=None
    ).add_to(marker_cluster)

# Добавляем слой для переключения между видами
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.LayerControl().add_to(m)

# Сохраняем карту
map_path = r"C:\Users\Qawse\Desktop\Кайрос\Спрос\tenders_distribution_map.html"
m.save(map_path)
print(f"Карта сохранена по пути: {map_path}")

Карта сохранена по пути: C:\Users\Qawse\Desktop\Кайрос\Спрос\tenders_distribution_map.html


# Работа с пропусками

In [24]:
import re

CATEGORY_PATTERNS = {
    "Чиллер": [r"\bчиллер(ы|ов|а|у|ом)?\b"],
    "Холодильный агрегат": [
        r"холодильн(ый|ого|ом|ым|ые|ых|ыми) агрегат",
        r"агрегат(ы)? холодильн(ый|ого|ом|ым|ые|ых|ыми)"
    ],
    "Компрессорно-конденсаторный блок": [r"компрессорно-конденсаторн(ый|ого|ом|ым|ые|ых|ыми) блок"],
    "Холодильная установка": [r"холодильн(ая|ой|ую|ые|ых|ыми) установк(а|и|у|ой|е|ами|ах)"]
}

def get_category(row):
    # Объединяем текст из обоих полей (если они есть)
    text = ""
    if pd.notna(row.get('reason')):
        text += str(row['reason']).lower() + " "
    if pd.notna(row.get('tender')):
        text += str(row['tender']).lower()
    
    for category, patterns in CATEGORY_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return category
    return "Другое"

# 2. Подготовка данных
df['end_date'] = pd.to_datetime(df['end_date'])
df['year'] = df['end_date'].dt.year
df['month'] = df['end_date'].dt.month
df['category'] = df.apply(get_category, axis=1)

In [25]:
# Убедимся, что страты созданы правильно
df['stratum'] = df['year'].astype(str) + '_' + df['month'].astype(str) + '_' + df['region_clean'] + '_' + df['category']

# Проверим, есть ли данные для расчёта медиан
if df[df['start_price'].notna()].empty:
    raise ValueError("Нет данных с указанными ценами для расчёта медиан")

# Рассчитаем медианы по стратам
median_prices = (
    df[df['start_price'].notna()]
    .groupby('stratum', as_index=False)
    .agg(median_price=('start_price', 'median'))
)

# Проверим результат объединения
df = df.merge(median_prices, on='stratum', how='left')
if 'median_price' not in df.columns:
    raise ValueError("Колонка 'median_price' не создана при объединении")

# Для страт с n < 5 используем резерв (регион + категория)
small_strata_mask = df.groupby('stratum')['start_price'].transform('count') < 5
df['fallback_stratum'] = df['region_clean'] + '_' + df['category']
fallback_median = df[df['start_price'].notna()].groupby('fallback_stratum')['start_price'].median()

# Импутация с проверками
df['fallback_median'] = df['fallback_stratum'].map(fallback_median)
overall_median = df['start_price'].median()

df['imputed_price'] = (
    df['start_price']
    .fillna(df['median_price'])
    .fillna(df['fallback_median'])
    .fillna(overall_median)
)

# Создаём отчёт по годам
yearly_stats = df.groupby('year').agg(
    Количество_контрактов=('imputed_price', 'size'),
    Контракты_с_ценой=('start_price', lambda x: x.notna().sum()),
    Суммарный_спрос=('imputed_price', 'sum')
).reset_index()

# Форматируем вывод
yearly_stats['Суммарный_спрос'] = yearly_stats['Суммарный_спрос'].apply(
    lambda x: f"{x:,.2f} руб.".replace(',', ' ')
)
yearly_stats['Доля_с_ценой'] = (yearly_stats['Контракты_с_ценой'] / yearly_stats['Количество_контрактов']).apply(
    lambda x: f"{x:.1%}"
)

# Красивое отображение
result_table = yearly_stats.rename(columns={
    'year': 'Год',
    'Количество_контрактов': 'Всего контрактов',
    'Контракты_с_ценой': 'Контрактов с ценой',
    'Доля_с_ценой': 'Доля с ценой'
})

print("Анализ спроса по годам:")
print(result_table[['Год', 'Всего контрактов', 'Контрактов с ценой', 'Доля с ценой', 'Суммарный_спрос']]
      .to_string(index=False))

Анализ спроса по годам:
 Год  Всего контрактов  Контрактов с ценой Доля с ценой        Суммарный_спрос
2020               545                 339        62.2% 22 815 625 087.36 руб.
2021               814                 536        65.8% 11 691 125 765.39 руб.
2022               658                 427        64.9% 10 198 708 754.13 руб.
2023               729                 470        64.5% 11 472 206 209.96 руб.
2024               709                 398        56.1% 34 728 522 447.69 руб.


In [26]:
category_by_year = df.groupby(['year', 'category']).size().unstack(fill_value=0)

# Транспонируем чтобы категории были строками, годы - столбцами
category_by_year_transposed = category_by_year.T

# Добавляем столбец с изменением за период в процентах
category_by_year_transposed['Изменение за период, %'] = ((category_by_year_transposed[2024] - category_by_year_transposed[2020]) / category_by_year_transposed[2020] * 100).round(1)

# Переименовываем индексы и столбцы для читаемости
category_by_year_transposed.index.name = 'Категория'
category_by_year_transposed.columns.name = 'Год'

# Отображаем таблицу
display(category_by_year_transposed)

Год,2020,2021,2022,2023,2024,"Изменение за период, %"
Категория,,,,,,
Другое,146,173,203,192,227,55.5
Компрессорно-конденсаторный блок,32,49,50,68,55,71.9
Холодильная установка,134,211,122,150,100,-25.4
Холодильный агрегат,70,94,66,72,69,-1.4
Чиллер,163,287,217,247,258,58.3


In [10]:
import pandas as pd
import numpy as np
from scipy import stats

def create_descriptive_stats_table(df, price_column, title):
    """
    Создает таблицу с описательными статистиками для научной статьи
    """
    # Базовые статистики
    stats_data = {
        'Показатель': [
            'Количество наблюдений',
            'Доля пропущенных значений',
            'Среднее значение',
            'Стандартное отклонение',
            'Медиана',
            'Минимальное значение',
            'Максимальное значение',
            '25-й перцентиль',
            '75-й перцентиль',
            'Коэффициент вариации (%)',
            'Коэффициент асимметрии',
            'Коэффициент эксцесса'
        ],
        'Значение': [
            len(df),
            f"{(df[price_column].isna().sum() / len(df) * 100):.1f}%",
            f"{df[price_column].mean():,.0f} руб.",
            f"{df[price_column].std():,.0f} руб.",
            f"{df[price_column].median():,.0f} руб.",
            f"{df[price_column].min():,.0f} руб.",
            f"{df[price_column].max():,.0f} руб.",
            f"{df[price_column].quantile(0.25):,.0f} руб.",
            f"{df[price_column].quantile(0.75):,.0f} руб.",
            f"{(df[price_column].std() / df[price_column].mean() * 100):.1f}%",
            f"{df[price_column].skew():.2f}",
            f"{df[price_column].kurtosis():.2f}"
        ]
    }
    
    stats_df = pd.DataFrame(stats_data)
    stats_df['Период'] = title
    
    return stats_df

# Создаем статистики ДО импутации
before_imputation = df[df['start_price'].notna()].copy()
stats_before = create_descriptive_stats_table(before_imputation, 'start_price', 'До импутации')

# Создаем статистики ПОСЛЕ импутации
stats_after = create_descriptive_stats_table(df, 'imputed_price', 'После импутации')

# Объединяем таблицы
comparison_table = pd.concat([stats_before, stats_after], ignore_index=True)

# Переформатируем для лучшего отображения
final_table = comparison_table.pivot(
    index='Показатель', 
    columns='Период', 
    values='Значение'
).reset_index()

print(final_table.to_string(index=False))

               Показатель        До импутации     После импутации
          25-й перцентиль        148,222 руб.        228,681 руб.
          75-й перцентиль      3,376,449 руб.      2,380,662 руб.
Доля пропущенных значений                0.0%                0.0%
    Количество наблюдений                2170                3455
   Коэффициент асимметрии               32.13               35.62
 Коэффициент вариации (%)             1049.1%             1115.3%
     Коэффициент эксцесса             1251.82             1635.02
    Максимальное значение 14,284,684,294 руб. 14,284,684,294 руб.
                  Медиана        517,913 руб.        517,913 руб.
     Минимальное значение              2 руб.              2 руб.
         Среднее значение     33,544,151 руб.     26,311,487 руб.
   Стандартное отклонение    351,910,261 руб.    293,461,420 руб.


In [11]:
seasonality_stats = df.groupby('month').agg(
    total_orders=('imputed_price', 'size'),
    total_demand=('imputed_price', 'sum'),
    avg_price=('imputed_price', 'mean'),
    median_price=('start_price', 'median')  # <-- ключевая правка
).reset_index()

# Добавляем названия месяцев
month_names = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
               'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
seasonality_stats['Месяц'] = seasonality_stats['month'].apply(lambda x: month_names[x-1])

# Форматируем таблицу
seasonality_stats['Суммарный спрос (руб.)'] = seasonality_stats['total_demand'].apply(lambda x: f"{x:,.2f}".replace(",", " "))
seasonality_stats['Средняя цена контракта (руб.)'] = seasonality_stats['avg_price'].apply(lambda x: f"{x:,.2f}".replace(",", " "))
seasonality_stats['Медианная цена контракта (руб.)'] = seasonality_stats['median_price'].apply(lambda x: f"{x:,.2f}".replace(",", " "))
seasonality_stats['Количество контрактов'] = seasonality_stats['total_orders']

seasonality_table = seasonality_stats[['Месяц', 'Количество контрактов',
                                       'Суммарный спрос (руб.)',
                                       'Средняя цена контракта (руб.)',
                                       'Медианная цена контракта (руб.)']]

print("Сезонность спроса (средняя цена — с учётом импутации, медиана — только по фактическим данным):")
print(seasonality_table.to_string(index=False))

Сезонность спроса (средняя цена — с учётом импутации, медиана — только по фактическим данным):
   Месяц  Количество контрактов Суммарный спрос (руб.) Средняя цена контракта (руб.) Медианная цена контракта (руб.)
  Январь                    146       2 005 846 659.82                 13 738 675.75                      388 390.04
 Февраль                    239       3 475 808 568.03                 14 543 132.08                      551 800.08
    Март                    299       4 638 151 678.35                 15 512 212.97                      504 676.96
  Апрель                    312       5 713 590 498.73                 18 312 790.06                      604 540.50
     Май                    254       2 924 917 845.91                 11 515 424.59                      477 000.00
    Июнь                    324      25 528 170 090.09                 78 790 648.43                      551 287.25
    Июль                    330       5 968 759 489.66                 18 087 149.97  

In [12]:
seasonality_table

,Месяц,Количество контрактов,Суммарный спрос (руб.),Средняя цена контракта (руб.),Медианная цена контракта (руб.)
0,Январь,146,2 005 846 659.82,13 738 675.75,388 390.04
1,Февраль,239,3 475 808 568.03,14 543 132.08,551 800.08
2,Март,299,4 638 151 678.35,15 512 212.97,504 676.96
3,Апрель,312,5 713 590 498.73,18 312 790.06,604 540.50
4,Май,254,2 924 917 845.91,11 515 424.59,477 000.00
5,Июнь,324,25 528 170 090.09,78 790 648.43,551 287.25
6,Июль,330,5 968 759 489.66,18 087 149.97,646 400.00
7,Август,271,5 958 179 482.24,21 985 902.15,584 380.88
8,Сентябрь,326,6 026 060 199.72,18 484 847.24,415 000.00
9,Октябрь,304,4 444 400 195.32,14 619 737.48,561 364.02


In [13]:
# 1. Посмотрим распределение реальных цен (только start_price, без импутации)
df[df['start_price'].notna()].groupby('month')['start_price'].describe()

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,88.0,2.170559e+07,1.321000e+08,2000.00,196664.192500,388390.045,1.659272e+06,1.164728e+09
2,133.0,2.289850e+07,1.192530e+08,8400.00,165000.000000,551800.080,2.267222e+06,1.097731e+09
3,178.0,2.119450e+07,9.718770e+07,11188.88,117000.000000,504676.960,3.881542e+06,1.069705e+09
4,194.0,1.319939e+07,6.307502e+07,5030.00,189697.912500,604540.500,3.209167e+06,6.026647e+08
5,153.0,1.551429e+07,6.942229e+07,7091.68,165500.000000,477000.000,4.299256e+06,5.908778e+08
6,204.0,1.105674e+08,1.021613e+09,4441.70,122074.625000,551287.250,4.570371e+06,1.428468e+10
7,215.0,2.631251e+07,1.292048e+08,3950.00,171800.000000,646400.000,7.494499e+06,1.612198e+09
8,171.0,1.814328e+07,8.861680e+07,2080.00,167285.825000,584380.880,3.975000e+06,8.692123e+08
9,215.0,2.631596e+07,1.282532e+08,2.00,118875.000000,415000.000,2.802584e+06,1.303532e+09


In [14]:
seasonality_table

,Месяц,Количество контрактов,Суммарный спрос (руб.),Средняя цена контракта (руб.),Медианная цена контракта (руб.)
0,Январь,146,2 005 846 659.82,13 738 675.75,388 390.04
1,Февраль,239,3 475 808 568.03,14 543 132.08,551 800.08
2,Март,299,4 638 151 678.35,15 512 212.97,504 676.96
3,Апрель,312,5 713 590 498.73,18 312 790.06,604 540.50
4,Май,254,2 924 917 845.91,11 515 424.59,477 000.00
5,Июнь,324,25 528 170 090.09,78 790 648.43,551 287.25
6,Июль,330,5 968 759 489.66,18 087 149.97,646 400.00
7,Август,271,5 958 179 482.24,21 985 902.15,584 380.88
8,Сентябрь,326,6 026 060 199.72,18 484 847.24,415 000.00
9,Октябрь,304,4 444 400 195.32,14 619 737.48,561 364.02


In [15]:
yearly_demand = df.groupby('year')['start_price'].sum().apply(lambda x: f"{x:,.2f} руб")
yearly_demand

year
2020    18,082,167,893.47 руб
2021     9,763,117,505.99 руб
2022     9,648,639,012.70 руб
2023     9,592,598,530.47 руб
2024    25,704,283,954.65 руб
Name: start_price, dtype: object

In [16]:
propuski = df.groupby('year')['start_price'].apply(lambda x: x.isna().sum())
propuski

year
2020    206
2021    278
2022    231
2023    259
2024    311
Name: start_price, dtype: int64

In [27]:
gdp_data_rub = {
    'year': [2020, 2021, 2022, 2023, 2024],
    'gdp_bln_rub': [107658.1, 134727.5, 156940.9, 176413.6, 201152.1]
}

procurement_data = {
    'year': [2020, 2021, 2022, 2023, 2024],
    'total_procurement_bln_rub': [28800.0, 22900.0, 27793.6, 34891.7, 35308.5]
}

refrigerator_procurement = {
    'year': [2020, 2021, 2022, 2023, 2024],
    'refrigerator_procurement': [18082167893.47, 9763117505.99, 9648639012.70, 
                                 9592598530.47, 25704283954.65]
}

df = pd.DataFrame(gdp_data_rub)
df = df.merge(pd.DataFrame(procurement_data), on='year')
df = df.merge(pd.DataFrame(refrigerator_procurement), on='year')

df['estimated_real_demand'] = df['refrigerator_procurement'] / (28.5 / 100)

result_df = df[['year', 'refrigerator_procurement', 'estimated_real_demand', 'gdp_bln_rub']].copy()
result_df['Объём открытых закупок, млрд руб.'] = result_df['refrigerator_procurement'] / 1e9
result_df['Оценённый совокупный спрос, млрд руб.'] = result_df['estimated_real_demand'] / 1e9
result_df['Номинальный ВВП, млрд руб.'] = result_df['gdp_bln_rub']

final_df = result_df[['year', 'Объём открытых закупок, млрд руб.', 'Оценённый совокупный спрос, млрд руб.', 'Номинальный ВВП, млрд руб.']]
final_df.columns = ['Год', 'Объём открытых закупок, млрд руб.', 'Оценённый совокупный спрос, млрд руб.', 'Номинальный ВВП, млрд руб.']

# Добавляем строку с изменением за период в процентах
change_zakupki = ((final_df.iloc[-1]['Объём открытых закупок, млрд руб.'] - final_df.iloc[0]['Объём открытых закупок, млрд руб.']) / final_df.iloc[0]['Объём открытых закупок, млрд руб.'] * 100).round(1)
change_spros = ((final_df.iloc[-1]['Оценённый совокупный спрос, млрд руб.'] - final_df.iloc[0]['Оценённый совокупный спрос, млрд руб.']) / final_df.iloc[0]['Оценённый совокупный спрос, млрд руб.'] * 100).round(1)
change_vvp = ((final_df.iloc[-1]['Номинальный ВВП, млрд руб.'] - final_df.iloc[0]['Номинальный ВВП, млрд руб.']) / final_df.iloc[0]['Номинальный ВВП, млрд руб.'] * 100).round(1)

final_df.loc[len(final_df)] = ['Изменение за период, %', f'{change_zakupki}%', f'{change_spros}%', f'{change_vvp}%']

display(final_df)

C:\Users\Qawse\AppData\Local\Temp\ipykernel_18768\3246172773.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[len(final_df)] = ['Изменение за период, %', f'{change_zakupki}%', f'{change_spros}%', f'{change_vvp}%']


,Год,"Объём открытых закупок, млрд руб.","Оценённый совокупный спрос, млрд руб.","Номинальный ВВП, млрд руб."
0,2020,18.082168,63.446203,107658.1
1,2021,9.763118,34.256553,134727.5
2,2022,9.648639,33.854874,156940.9
3,2023,9.592599,33.65824,176413.6
4,2024,25.704284,90.19047,201152.1
5,"Изменение за период, %",42.2%,42.2%,86.8%
